In [ ]:
"""
Created on Mon May 02 11:15 2022

Correlation matrix input data

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import trange, tqdm
import seaborn as sns

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'
outputpath_nn_models = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/'
outputpath_doc = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/custom_doc/'
plot_path = '/bettik/burgardc/PLOTS/NN_plots/input_vars/'

In [ ]:
input_dataset = 'valid' #valid, train
if input_dataset == 'train':
    run_list = ['OPM006', 'OPM016', 'OPM018', 'OPM031-2'] #training dataset
elif input_dataset == 'valid':
    run_list = ['OPM021', 'OPM026', 'OPM027', 'OPM031-1'] #validationa dataset

In [ ]:
TS_input = 'extrapolated' # 'whole_prof'extrapolated

In [ ]:
isf_list = [10, 11, 12, 13, 18, 21, 22, 23, 24, 25, 30, 31, 33, 38, 39, 40, 42, 43, 44, 45, 47, 48, 51, 52, 53, 54, 55, 58, 61, 65, 66, 69, 70, 71, 73, 75]

In [ ]:
if TS_input == 'whole_prof':
    csv_path = inputpath_data + 'WHOLE_PROF/'
elif TS_input == 'extrapolated':
    csv_path = inputpath_data + 'EXTRAPOLATED_ISFDRAFT/'

all_input_df = None        
    
for nemo_run in run_list:
    
    for kisf in tqdm(isf_list): 
    #for kisf in tqdm([23,66]): 
    #for kisf in tqdm([23]): 
        clean_df_nrun_kisf = pd.read_csv(csv_path + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+nemo_run+'.csv',index_col=[0,1,2])
        clean_df_nrun_kisf.reset_index(drop=True, inplace=True)
        clean_ds_nrun_kisf = clean_df_nrun_kisf.to_xarray()

        if all_input_df is None:
            all_input_df = clean_ds_nrun_kisf.copy()
        else:
            #print(clean_ds_nrun_kisf.index)
            #print(all_input_df.index.max())
            new_index = clean_ds_nrun_kisf.index.values + all_input_df.index.max().values+1
            #print(new_index)
            clean_ds_nrun_kisf = clean_ds_nrun_kisf.assign_coords({'index': new_index})
            all_input_df = xr.concat([all_input_df, clean_ds_nrun_kisf], dim='index') 

#all_input_df.to_netcdf(new_path_input + 'dataset_input_'+timetag+'.nc','w')

In [ ]:
corr_matrix = all_input_df.to_dataframe().corr()

In [ ]:
corr_matrix_round = corr_matrix.round(1)

In [ ]:
corr_matrix_round

In [ ]:
if TS_input == 'whole_prof':
    f = plt.figure(figsize=(8.24*8,8.24*8))
elif TS_input == 'extrapolated':
    f = plt.figure(figsize=(8.24,8.24))
sns.heatmap(corr_matrix_round, annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
f.savefig(plot_path+'cross_correlation_input_'+TS_input+'_'+input_dataset+'.png')

In [ ]:
TS_input


In [ ]:
input_dataset